In [1]:
import pandas as pd
import re
from collections import Counter
import string

In [2]:
preText = ''
with open('plant.wsd', 'r') as file:
    preText = file.read()
inputData = preText.strip().split("\n\n")
df = pd.DataFrame(inputData)
# df[0][1] = df[0][1].strip().split("\n")
inputDataDF = df.apply (lambda x: x[0].strip().split("\n"), axis = 1)
# inputDataDF

In [3]:
inputDataList  = inputDataDF.to_list()
len(inputDataList)

188

In [4]:
# Creating the five folds:
import math
def creatingFiveFolds(inputList):
    folds = []
    totalInstances = int(math.ceil(len(inputList)*0.1)/0.1)
    # if (totalInstances % 5 == 0):
    # if (totalInstances%5)
    indices = [ int(totalInstances/5 + i*totalInstances/5) for i in range(5)]
    folds.append(inputList[0: indices[0]])
    folds.append(inputList[indices[0]: indices[1]])
    folds.append(inputList[indices[1]: indices[2]])
    folds.append(inputList[indices[2]: indices[3]])
    folds.append(inputList[indices[3]: indices[4]])
    folds.append(inputList[indices[4]: ])
    return folds

folds = creatingFiveFolds(inputDataList)

In [5]:
noInstances = len(inputDataDF)
noInstances

188

In [6]:
# function to get sense from the raw instance to a df for senses
def extractingTheSense(instanceRawText):
    answerInstance = instanceRawText[1].split(" ")
    sense_reg1 = re.search('\"(.*)\"', answerInstance[2])
    sense1 = sense_reg1.group(1)
    sense_reg2 = re.search('%(.*)', sense1)
    sense2 = sense_reg2.group(1)
    return sense2
print(extractingTheSense(inputDataList[2]))

living


In [7]:
def extractingTheID(instanceRawText):
    answerInstance = instanceRawText[1].split(" ")
    ID_reg1 = re.search('\"(.*)\"', answerInstance[1])
    ID1 = ID_reg1.group(1)
    ID_reg2 = re.search('\.(.*)', ID1)
    ID2 = ID_reg2.group(1)
    return ID2
print(extractingTheID(inputDataList[2]))

1000002


In [8]:
inputData[0]

'<instance id="plant.1000000" docsrc = "BNC/A07">\n<answer instance="plant.1000000" senseid="plant%factory"/>\n<context>\nUntil the mid- and late 1970s, there were simply no rules whereby groupings of parents could obtain a state-financed, multi-denominational school, as the only channels of communication in these matters were those between the Department of Education and the relevant diocesan department or other church board. For example, a new housing estate would be built, the diocese would be informed of the development at the planning stage, and the diocesan office would put in for a school, or an extension for an existing school. The arrangement had been unchanged since the early years of the state and had become entirely natural. Similarly for the protestant community: the school  <head>plant</head>  is owned by the church, or appropriate church body. \n</context>\n</instance>'

In [9]:
def extractingTheContext(instanceRawText):
    listOfWords = instanceRawText[3].split(" ")
    listOfWords = list(map(lambda x: re.sub(r'[^\w\s]','',x), listOfWords))
    listOfWords = list(filter(lambda x: x != '', listOfWords))
    return listOfWords
print(extractingTheContext(inputDataList[2]))

['September', '1991', '130', 'You', 'can', 'win', 'a', 'great', 'new', 'patio', 'Pippa', 'Wood', 'How', 'to', 'cope', 'with', 'a', 'slope', 'Bulbs', 'headplanthead', 'now', 'for', 'spring', 'blooms']


In [10]:
# Processing senses and counts
sensesList = list(map(lambda x: extractingTheSense(x), inputDataList))
uniqueSenses = list(set(sensesList))
sensesCountDict = dict(Counter(sensesList))
sensesCountDict
senseWordsDict = {key: {} for key in uniqueSenses} 
senseWordsDict

{'factory': {}, 'living': {}}

In [11]:
# processing the bag of words
# inputDataList[0][3]
def creatingBagOfWords(instanceRawText, senseWordCountDict, sensesDict, wordSet):
    sense = extractingTheSense(instanceRawText)
    # print(sensesDict)
    sensesDict[sense] += 1
    listOfWords = extractingTheContext(instanceRawText)
    
    for word in listOfWords:
        wordSet.add(word)
        if word in senseWordCountDict[sense].keys():
            senseWordCountDict[sense][word] += 1
        else:
            senseWordCountDict[sense][word] = 1   
    return senseWordCountDict
# print(creatingBagOfWords(inputDataList[0], senseWordsDict))
inputDataList[0]


['<instance id="plant.1000000" docsrc = "BNC/A07">',
 '<answer instance="plant.1000000" senseid="plant%factory"/>',
 '<context>',
 'Until the mid- and late 1970s, there were simply no rules whereby groupings of parents could obtain a state-financed, multi-denominational school, as the only channels of communication in these matters were those between the Department of Education and the relevant diocesan department or other church board. For example, a new housing estate would be built, the diocese would be informed of the development at the planning stage, and the diocesan office would put in for a school, or an extension for an existing school. The arrangement had been unchanged since the early years of the state and had become entirely natural. Similarly for the protestant community: the school  <head>plant</head>  is owned by the church, or appropriate church body. ',
 '</context>',
 '</instance>']

In [12]:
# applying the function on all the four trainings
# maybe remove the word we are looking into?
# remove stop words
# modify the training to rotate the folds
train = folds[:-1]
noTotalInstances = 0
sensesCountDict = dict.fromkeys((uniqueSenses), 0)
senseWordsDict = {key: {} for key in uniqueSenses} 
uniqueWordSet = set()
for fold in train:
    for instance in fold:
        noTotalInstances += 1
        creatingBagOfWords(instance, senseWordsDict, sensesCountDict, uniqueWordSet)
# test = folds[4]
# senseWordsDict['living']
# sensesCountDict
len(uniqueWordSet)

5676

In [13]:
# working in the test fold
# I need the ID, Context (list of words)
# Id and the gold standard senses
test 
sensesTestList = list(map(lambda x: extractingTheSense(x), test))
IDTestList = list(map(lambda x: extractingTheID(x), test))
contextTestList  = list(map(lambda x: extractingTheContext(x), test))
processedListTesting = list(zip(IDTestList, sensesTestList, contextTestList))
# processedListTesting[1]
# (sensesTestList)

NameError: name 'test' is not defined

In [ ]:
def calculateSenseProb (prob_sense, sense_count_dict, total):
    for sense in prob_sense.keys():
        prob_sense[sense] = float(sense_count_dict[sense] / total)

In [ ]:
def calculateWordSenseProb(probWordSenseDict, contextList, senseWord, v, sensesCount):
    for sense in sensesCount.keys():
        for word in contextList:
            if word in senseWord[sense].keys():
                probWordSenseDict[word] = {sense : float((senseWord[sense][word] + 1)/(sensesCount[sense] + v))}
            else:
                probWordSenseDict[word] = {sense : float( 1/(sensesCount[sense] + v))}

In [ ]:
def calculatingSenseForInstance(probSenseWordPred, contextWordList, senseList, probSense, probWordSense):
    for sense in senseList:
        for word in contextWordList:
            probSenseWordPred[sense] += 1 # math.log10(probWordSense[word][sense])
            if sense not in probWordSense[word].keys():
                print(word)
        probSenseWordPred[sense] += math.log10(probSense[sense])

In [ ]:
# doing the Naive Based Classification   
def naiveBasedClassifier(ListTesting, senseCountDict,senseWordsDict):
    # calculating the probablitiy of sense 
    # P(Sense)
    probSenseDict = dict.fromkeys((uniqueSenses), 0)
    calculateSenseProb(probSenseDict,senseCountDict,noTotalInstances)
    
    # find the probablity of words given sense, applying add one smoothing
    # P(feature|sense)]
    V = len(uniqueWordSet)
    probWordSenseDict = {}
    probSenseWordPred = dict.fromkeys((uniqueSenses), 0)
    for instance in ListTesting:
        contextWordList = instance[2]
        calculateWordSenseProb(probWordSenseDict, contextWordList, senseWordsDict, V, sensesCountDict)

        # Calculating the sense given word context
        calculatingSenseForInstance(probSenseWordPred, contextWordList, sensesList, probSenseDict, probWordSenseDict)
    return probWordSenseDict
# probWordSenseDict
x = naiveBasedClassifier(processedListTesting, sensesCountDict, senseWordsDict)
x

In [ ]:
probWordSenseDict